In [14]:
import pandas as pd
import yfinance as yf
import sqlite3
import pandas as pd
from datetime import datetime, timedelta

In [15]:
daftarSaham = pd.read_excel('Daftar Saham  - 20240829.xlsx')

In [16]:
# Define a mapping from Indonesian month names to English
month_map = {
    "Jan": "Jan",
    "Feb": "Feb",
    "Mar": "Mar",
    "Apr": "Apr",
    "Mei": "May",
    "Jun": "Jun",
    "Jul": "Jul",
    "Agt": "Aug",
    "Sep": "Sep",
    "Okt": "Oct",
    "Nov": "Nov",
    "Des": "Dec"
}

# Replace the month names in the 'IPO_Date' column
for ind_month, eng_month in month_map.items():
    daftarSaham['Tanggal Pencatatan'] = daftarSaham['Tanggal Pencatatan'].str.replace(ind_month, eng_month)

daftarSaham['Tanggal Pencatatan'] = pd.to_datetime(daftarSaham['Tanggal Pencatatan'])
daftarSaham['Kode'] = daftarSaham['Kode'] + '.JK'

In [19]:
# Example of how to use it
# stock = yf.Ticker('AALI.JK')
# data = stock.history(start='2019-08-29', end=datetime.today())
# data.reset_index(inplace=True)
# data.head()

In [31]:
def fetch_and_store_data(stock_df):

    end_date = datetime.today()
    start_date = end_date - timedelta(days=5*365)

    for index, row in stock_df.iterrows():
        ticker = row['Kode']
        ipo_date = row['Tanggal Pencatatan']
        
        # Determine the actual start date
        fetch_start_date = max(ipo_date, start_date)
        print(f"Fetching data for {ticker} from {fetch_start_date} to {end_date}...")
        
        # Fetch stock price data from the start date to today
        stock = yf.Ticker(ticker)
        data = stock.history(start=fetch_start_date, end=end_date)
        
        data.reset_index(inplace=True)

        # Insert stock price data into SQLite
        for row in data.itertuples():
            cursor.execute('''
                INSERT INTO stock_data (stock, date, open, high, low, close, volume)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (
                str(ticker), 
                str(row.Date), 
                float(row.Open) if pd.notnull(row.Open) else None, 
                float(row.High) if pd.notnull(row.High) else None, 
                float(row.Low) if pd.notnull(row.Low) else None, 
                float(row.Close) if pd.notnull(row.Close) else None, 
                int(row.Volume) if pd.notnull(row.Volume) else None
            ))

        # Fetch financial ratios
        pe_ratio = stock.info.get('trailingPE')
        debt_to_equity = stock.info.get('debtToEquity')
        pb_ratio = stock.info.get('priceToBook')
        roe = stock.info.get('returnOnEquity')
        trailing_eps = stock.info.get('trailingEps')
        forward_eps = stock.info.get('forwardEps')
        dividend_yield = stock.info.get('dividendYield')
        market_cap = stock.info.get('marketCap')
        beta = stock.info.get('beta')
        current_ratio = stock.info.get('currentRatio')
        quick_ratio = stock.info.get('quickRatio')
        free_cash_flow = stock.info.get('freeCashflow')

        print(f"Fetching ratios for {ticker}: P/E Ratio={pe_ratio}, Debt-to-Equity Ratio={debt_to_equity}, P/B Ratio={pb_ratio}, ROE={roe}, Dividend Yield={dividend_yield}")

        # Insert financial ratios into SQLite
        cursor.execute('''
            INSERT INTO ratios (stock, pe_ratio, debt_to_equity, pb_ratio, roe, trailing_eps, forward_eps, dividend_yield, market_cap, beta, current_ratio, quick_ratio, free_cash_flow)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            str(ticker), 
            float(pe_ratio) if pd.notnull(pe_ratio) else None, 
            float(debt_to_equity) if pd.notnull(debt_to_equity) else None,
            float(pb_ratio) if pd.notnull(pb_ratio) else None,
            float(roe) if pd.notnull(roe) else None,
            float(trailing_eps) if pd.notnull(trailing_eps) else None,
            float(forward_eps) if pd.notnull(forward_eps) else None,
            float(dividend_yield) if pd.notnull(dividend_yield) else None,
            float(market_cap) if pd.notnull(market_cap) else None,
            float(beta) if pd.notnull(beta) else None,
            float(current_ratio) if pd.notnull(current_ratio) else None,
            float(quick_ratio) if pd.notnull(quick_ratio) else None,
            float(free_cash_flow) if pd.notnull(free_cash_flow) else None
        ))
        
       
        conn.commit()


conn = sqlite3.connect('stocks.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS stock_data (
        stock TEXT,
        date TEXT,
        open REAL,
        high REAL,
        low REAL,
        close REAL,
        volume INTEGER
    )
''')

# Create the ratios table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS ratios (
        stock TEXT,
        pe_ratio REAL,
        debt_to_equity REAL,
        pb_ratio REAL,
        roe REAL,
        trailing_eps REAL,
        forward_eps REAL,
        dividend_yield REAL,
        market_cap REAL,
        beta REAL,
        current_ratio REAL,
        quick_ratio REAL,
        free_cash_flow REAL
    )
''')

fetch_and_store_data(daftarSaham)

# Close the database connection when done
conn.close()


Fetching data for AALI.JK from 2019-08-31 15:17:45.469783 to 2024-08-29 15:17:45.469783...
Fetching ratios for AALI.JK: P/E Ratio=9.829938, Debt-to-Equity Ratio=18.275, P/B Ratio=0.52656925, ROE=0.05509, Dividend Yield=0.0403
Fetching data for ABBA.JK from 2019-08-31 15:17:45.469783 to 2024-08-29 15:17:45.469783...
Fetching ratios for ABBA.JK: P/E Ratio=None, Debt-to-Equity Ratio=None, P/B Ratio=6.549232, ROE=None, Dividend Yield=None
Fetching data for ABDA.JK from 2019-08-31 15:17:45.469783 to 2024-08-29 15:17:45.469783...
Fetching ratios for ABDA.JK: P/E Ratio=32.57329, Debt-to-Equity Ratio=None, P/B Ratio=1.8387159, ROE=0.05652, Dividend Yield=None
Fetching data for ABMM.JK from 2019-08-31 15:17:45.469783 to 2024-08-29 15:17:45.469783...
Fetching ratios for ABMM.JK: P/E Ratio=None, Debt-to-Equity Ratio=132.03, P/B Ratio=16598.639, ROE=0.31846002, Dividend Yield=0.0634
Fetching data for ACES.JK from 2019-08-31 15:17:45.469783 to 2024-08-29 15:17:45.469783...
Fetching ratios for ACES.

In [26]:
# Testing
cursor.execute("SELECT * FROM stock_data LIMIT 10")
results = cursor.fetchall()

results_df = pd.DataFrame(results, columns=['Stock', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume'])

print(results_df)


     Stock                       Date         Open         High          Low  \
0  AALI.JK  2019-09-02 00:00:00+07:00  9027.504883  9156.469238  9027.504883   
1  AALI.JK  2019-09-03 00:00:00+07:00  9027.503990  9199.456447  9027.503990   
2  AALI.JK  2019-09-04 00:00:00+07:00  9048.997795  9048.997795  8963.021568   
3  AALI.JK  2019-09-05 00:00:00+07:00  9006.009512  9048.997624  8941.527344   
4  AALI.JK  2019-09-06 00:00:00+07:00  8941.527344  9006.009512  8898.539232   
5  AALI.JK  2019-09-09 00:00:00+07:00  8941.528155  9006.010329  8898.540039   
6  AALI.JK  2019-09-10 00:00:00+07:00  8898.540527  9070.493001  8898.540527   
7  AALI.JK  2019-09-11 00:00:00+07:00  9091.987061  9113.481120  8963.022705   
8  AALI.JK  2019-09-12 00:00:00+07:00  9113.479532  9113.479532  8877.044922   
9  AALI.JK  2019-09-13 00:00:00+07:00  8877.045425  9091.985993  8877.045425   

         Close  Volume  
0  9027.504883  598600  
1  9048.998047  793300  
2  8984.515625  309500  
3  8941.527344  553

In [27]:
# We are finsh time to close it
conn.close()